# Main Notebook

#### download weight and libraries


In [1]:
import os

weight_folder = './data/'
lib_folder = './src/'
img_folder = './img/'
if not os.path.exists(weight_folder): 
  os.mkdir(weight_folder)
if not os.path.exists(img_folder): 
  os.mkdir(img_folder)
if not os.path.exists(lib_folder): 
  os.mkdir(lib_folder)

# Download weight
weight_url = 'http://www.vlfeat.org/matconvnet/models/imagenet-vgg-verydeep-19.mat'
weight_path = weight_folder + 'imagenet-vgg-verydeep-19.mat'
if not os.path.exists(weight_path):
  !curl -o $weight_path $weight_url

# Download libraries
libraries = ['vgg.py', 'constants.py', 'neural_network.py', 'utils.py']
library_url = 'https://raw.githubusercontent.com/abx67/AdvML-style-transfer/master/src/'

for lib in libraries:
  lib_path = lib_folder + lib
  lib_url = library_url + lib
  if not os.path.exists(lib_path):
    !curl -o $lib_path $lib_url
    
# Download images
style_img_name = 'van_gogh.jpg'
content_img_name = 'new_york.jpg'
img_url = 'https://raw.githubusercontent.com/abx67/AdvML-style-transfer/master/img/'

style_path = img_folder + style_img_name
style_url = img_url + style_img_name
content_path = img_folder + content_img_name
content_url = img_url + content_img_name

if not os.path.exists(style_path):
  !curl -o $style_path $style_url
if not os.path.exists(content_path):
  !curl -o $content_path $content_url

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  510M  100  510M    0     0  63.5M      0  0:00:08  0:00:08 --:--:-- 67.3M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2470  100  2470    0     0  20081      0 --:--:-- --:--:-- --:--:-- 19919
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   779  100   779    0     0   7018      0 --:--:-- --:--:-- --:--:--  7018
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  7101  100  7101    0     0  58685      0 --:--:-- --:--:-- --:--:-- 58685
  % Total    % Received % Xferd  Average Speed   Tim

#### import modules

In [0]:
import time
import numpy as np
import PIL.Image
import sys
sys.path.append(lib_folder)
from neural_network import NeuralNetwork
from vgg import VGG

#### contents definition

In [0]:
VGG_MAT_PATH = weight_path
CONTENT_IMAGE_PATH = content_path
STYLE_IMAGE_PATH = style_path
MIXED_IMAGE_PATH = './output/'
if not os.path.exists(MIXED_IMAGE_PATH): 
  os.mkdir(MIXED_IMAGE_PATH)

CONTENT_WEIGHT = 5
STYLE_WEIGHT = 500
VARIATION_WEIGHT = 100
LEARNING_RATE = 10
BETA1 = 0.9
BETA2 = 0.999
EPSILON = 1e-08
MAX_ITERATION = 100
POOLING = 'max'
CHECK_POINT = 10

#### function definition

In [0]:
def load_image(file_path, max_size=None, shape=None):
    # load image and define the factor used to tranfer the image size
    image = PIL.Image.open(file_path)
    # resize by max_size
    if max_size is not None:
        factor = float(max_size) / np.max(image.size)  # image.size = [height, width, 3]
        size = np.array(image.size) * factor
        size = size.astype(int)
        image = image.resize(size, PIL.Image.LANCZOS)  # image resize with filter LANCZOS
    # resize with shape
    if shape is not None:
        image = image.resize(shape, PIL.Image.LANCZOS)
    # return image values with float data type
    return np.float32(image)


def save_image(file_path, image):
    # ensure the pixel value is int between 0 and 255
    image = np.clip(image, 0.0, 255.0).astype(np.uint8)
    # write to file
    PIL.Image.fromarray(image).save(file_path)
    return


def style_transfer(content_image_path, style_image_path, mixed_image_path,
                   content_weight, style_weight, variation_weight,
                   pooling, learning_rate, beta1, beta2, epsilon, max_iteration, check_point):
    # set the time point
    time_start = time.time()

    # load image
    content_image = load_image(content_image_path)
    style_image = load_image(style_image_path, shape=content_image.shape[:2])

    # initialize object
    vgg = VGG(VGG_MAT_PATH, pooling)
    nn = NeuralNetwork(content_image, style_image, vgg, content_weight, style_weight, variation_weight)

    # train the model
    for i, mixed_image in nn.train_model(learning_rate, beta1, beta2, epsilon, max_iteration, check_point):
        save_image(mixed_image_path + 'v1_{}.jpeg'.format(i + 1), mixed_image)

    # print time
    time_end = time.time()
    print('Time elapsed: {} seconds'.format(round(time_end - time_start)))

    return

#### main function

In [5]:
if __name__ == '__main__':
    style_transfer(content_image_path=CONTENT_IMAGE_PATH,
                   style_image_path=STYLE_IMAGE_PATH,
                   mixed_image_path=MIXED_IMAGE_PATH,
                   content_weight=CONTENT_WEIGHT,
                   style_weight=STYLE_WEIGHT,
                   variation_weight=VARIATION_WEIGHT,
                   pooling=POOLING,
                   learning_rate=LEARNING_RATE,
                   beta1=BETA1,
                   beta2=BETA2,
                   epsilon=EPSILON,
                   max_iteration=MAX_ITERATION,
                   check_point=CHECK_POINT)

iter: 10, loss total: 7170729.5, loss content: 2416619.5, loss style: 4495219.0, loss variation: 258890.765625
iter: 20, loss total: 4637959.5, loss content: 2342415.0, loss style: 2018924.75, loss variation: 276619.6875
iter: 30, loss total: 3293829.0, loss content: 2181426.5, loss style: 872259.5625, loss variation: 240143.109375
iter: 40, loss total: 2827415.25, loss content: 2023538.5, loss style: 579251.8125, loss variation: 224624.953125
iter: 50, loss total: 2552220.0, loss content: 1897504.0, loss style: 434564.125, loss variation: 220151.90625
iter: 60, loss total: 2378583.25, loss content: 1805661.875, loss style: 358922.53125, loss variation: 213998.859375
iter: 70, loss total: 2259197.75, loss content: 1739959.25, loss style: 312821.375, loss variation: 206417.0625
iter: 80, loss total: 2174798.25, loss content: 1691426.625, loss style: 283187.65625, loss variation: 200184.109375
iter: 90, loss total: 2116538.25, loss content: 1653784.75, loss style: 267382.625, loss variat